# ETL YELP

# 1. Importar librerias

In [62]:
import pandas as pd
import numpy as np
import re
import requests
#from funciones import data_cleaned
import warnings
warnings.filterwarnings("ignore", message="Downcasting.*", category=FutureWarning)


# 2. Carga de Datos

In [64]:
dfBusinessYelp = pd.read_csv('DataSets/businessYelp.csv')


C:\Users\U ANGEL\AppData\Local\Temp\ipykernel_14180\2635447684.py:1: DtypeWarning: Columns (14,15,16,17,18,19,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBusinessYelp = pd.read_csv('DataSets/businessYelp.csv')


In [71]:
dfReviewYelp = pd.read_csv('DataSets/reviewsYelp.csv')

# 3. Exploracion y Limpieza de Datos

## 3.1 Dataset Business Yelp

### 3.1.1. Exploración inicial de los datos

In [65]:
#Vizualizo las primeras filas del DataFrame
dfBusinessYelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state.1,postal_code.1,latitude.1,longitude.1,stars.1,review_count.1,is_open.1,attributes.1,categories.1,hours.1
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# Obtener información sobre las columnas y los tipos de datos
dfBusinessYelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   business_id     150346 non-null  object 
 1   name            150346 non-null  object 
 2   address         145219 non-null  object 
 3   city            150346 non-null  object 
 4   state           150343 non-null  object 
 5   postal_code     150273 non-null  object 
 6   latitude        150346 non-null  float64
 7   longitude       150346 non-null  float64
 8   stars           150346 non-null  float64
 9   review_count    150346 non-null  int64  
 10  is_open         150346 non-null  int64  
 11  attributes      136602 non-null  object 
 12  categories      150243 non-null  object 
 13  hours           127123 non-null  object 
 14  business_id.1   5 non-null       object 
 15  name.1          5 non-null       object 
 16  address.1       5 non-null       object 
 17  city.1    

In [67]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfBusinessYelp.isnull().sum()
print(filas_vacias_por_columna)

business_id            0
name                   0
address             5127
city                   0
state                  3
postal_code           73
latitude               0
longitude              0
stars                  0
review_count           0
is_open                0
attributes         13744
categories           103
hours              23223
business_id.1     150341
name.1            150341
address.1         150341
city.1            150341
state.1           150341
postal_code.1     150341
latitude.1        150341
longitude.1       150341
stars.1           150341
review_count.1    150341
is_open.1         150341
attributes.1      150341
categories.1      150341
hours.1           150341
dtype: int64


Al evaluar la cantidad de nulos por columnas vemos que las columnas : 'business_id.1', 'name.1',
       'address.1', 'city.1', 'state.1', 'postal_code.1', 'latitude.1',
       'longitude.1', 'stars.1', 'review_count.1', 'is_open.1', 'attributes.1',
       'categories.1', 'hours.1', estan llenos de nulos

In [68]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfBusinessYelp.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


Como logramos observas no encontramos valores duplicados en el DataFrame

In [69]:
dfBusinessYelp['state'].unique()

array([nan, 'CA', 'MO', 'AZ', 'PA', 'TN', 'FL', 'IN', 'LA', 'AB', 'NV',
       'ID', 'DE', 'IL', 'NJ', 'NC', 'CO', 'WA', 'HI', 'UT', 'TX', 'MT',
       'MI', 'SD', 'XMS', 'MA', 'VI', 'VT'], dtype=object)

Al ver que hay valores nulos, siglas de los diferentes estados concluyo que la columna 'state' es mejor no usarla y sacar los estados mediante la longitud y latitud

### 3.1.2. Limpieza y Normalización de datos

#### Filtrado de datos

In [73]:
# El alcance del proyecto se limita al negocio llamado Subway, por lo que se aplica el filtro.

# Aplicamos el filtro con referencia en la columna 'name'
dfBusinessYelp = dfBusinessYelp[dfBusinessYelp['name'] == 'Subway']

De limitamos a una franquicia de comida rapida 'Subway'(sandwiches) que hemos puesto como objetivo y delimitador

#### Limpieza y Normalizacion

In [76]:
columnas_a_eliminar = ['is_open','address', 'name', 'categories','state', 'attributes', 
       'hours','business_id.1', 'name.1', 'address.1', 'city.1', 'state.1', 
       'postal_code.1', 'latitude.1', 'longitude.1', 'stars.1', 'review_count.1',
        'is_open.1', 'attributes.1', 'categories.1', 'hours.1']

# Eliminar las columnas especificadas
dfBusinessYelp = dfBusinessYelp.drop(columnas_a_eliminar, axis=1)

Como mencionamos en el conteo de nulos eliminamos columnas con demasiados valores vacios que no nos ayudaran para realizar un buen analisis.
Las columnas'is_open','address', 'name' ya tenemos los valores y se filtro de manera correcta asi que seria redundante tener esas columnnas.
Con respecto a la columna 'state' tiene valores que no corresponde con la latitud y longitud en el mapa, asi que mejor es no contar con esa columna.

In [81]:
# Define una función para obtener el estado desde la latitud y longitud
def reverse_geocode(lat, lon):
    # Aquí va tu código para obtener el estado desde la latitud y longitud
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}"
    response = requests.get(url)
    data = response.json()
    if 'address' in data:
        state = data['address'].get('state', '')
        return state

# Aplica la función a cada fila del DataFrame y crea una lista de resultados
states = [reverse_geocode(row['latitude'], row['longitude']) for _, row in dfBusinessYelp.iterrows()]

# Asigna la lista de estados a la columna 'state' del DataFrame
dfBusinessYelp['state'] = states

Usando la API OpenStreetMap (OSM) se realizo una geocodificacion inversa que me devuelve los diferentes estados('state') basado en las columnas 'latitude' y 'longitude' 

In [84]:
# Se creó la columna 'source' con un valor unico 'Y' 
dfBusinessYelp['source']='Y'

Se creo la colummna 'source' para identificar la tabla que pertenecen a los datos obtenidos de las tablas Yelp

In [87]:
nuevo_orden_columnas = ['business_id', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count']

dfBusinessYelp = dfBusinessYelp[nuevo_orden_columnas]

In [88]:
dfBusinessYelp.head()

,business_id,city,state,postal_code,latitude,longitude,stars,review_count
122,VZzivJEbmHItOxNXJB3SpA,Brentwood,Tennessee,37027,36.026363,-86.789473,2.5,14
964,vuL3QCjFZ7lU5LMdUAmZ9Q,Wayne,Pennsylvania,19087,40.044263,-75.399354,1.5,5
1417,Hqz96v1ymucUKNzIWfEKXw,Santa Barbara,California,93101,34.430822,-119.714156,3.0,5
1649,gi90NMmRhzl8vzM83UNOVA,Tampa,Florida,33647,28.150110,-82.353997,3.0,14
1656,qZIiURCjON7QcTzNtG9JpQ,Nashville,Tennessee,37205,36.131607,-86.858115,1.5,8


## 3.2 Dataset Reviews Yelp

### 3.2.1. Exploración inicial de los datos

In [16]:
#Vizualizo las primeras filas del DataFrame
dfReviewYelp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,F6VdYuJiefNBfn3HNELv0A,s4sR0rvVOf6iby77xGeyLg,z7em5co2qckbAXoDGXynsA,5,0,0,0,The food is INCREDIBLE! We didn't have time to...,2019-01-04 02:18:09
1,nAMDCKElSKxOhzm9Lpt6Eg,SRp90x9d2719GOZ_PT-a6A,M0r9lUn2gLFYgIwIfG8-bQ,5,0,0,0,"We had a great time, and excellent service. Al...",2019-01-06 11:48:21
2,3CmdoGKBZUX3Nb5IfbztMg,IDOQnhI1GOkLIDaTbBxkfw,itAhmbhHOyQQparfwicjDQ,5,0,0,0,My favorite coffee shop in New Orleans for sur...,2019-01-27 15:08:14
3,-4Nv_JAolCM0gzKM4DZpmQ,NjuPidqqsNW9KG8PBPrycg,-3AooxIkg38UyUdlz5oXdw,4,2,0,1,"Old school circa 1979, the cozy, intimate banq...",2019-02-17 20:28:26
4,_eQuEUgJkgHQjKEO00ZK9Q,X71-eKfjEgfSxApe2MtrfQ,TVGuOv0Nc0omLXtGKVOwdQ,1,0,0,0,"All Lowes in 20 mile radius, stopped\nstocking...",2019-02-17 17:17:56


In [17]:
dfReviewYelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2078061 entries, 0 to 2078060
Data columns (total 9 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   user_id      object
 2   business_id  object
 3   stars        int64 
 4   useful       int64 
 5   funny        int64 
 6   cool         int64 
 7   text         object
 8   date         object
dtypes: int64(4), object(5)
memory usage: 142.7+ MB


In [18]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfReviewYelp.isnull().sum()
print(filas_vacias_por_columna)

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64


Como observamos no tenemos valores nulos

In [19]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfReviewYelp.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


Como observamos no tenemos filas duplicadas

### 3.2.2. Limpieza y Normalización de datos

Filtrado de datos

In [70]:
# Obtener los id únicos del DataFrame dfbusinessYelp
business_ids_to_keep = dfBusinessYelp['business_id'].unique()


# Friltamos el dfreviewYelp basado en los balores unico de lao id de dfbusinessYelp
dfReviewYelp = dfReviewYelp[dfReviewYelp['business_id'].isin(business_ids_to_keep)]

# El conjunto de datos se filtró por la fecha desde el inicio del 2016 hasta finalizar el 2021.
#dfreviewYelp = dfreviewYelp[(dfreviewYelp['date'] >= '2016-01-01') & (dfreviewYelp['date'] >= '2021-12-31')]

Aplicamos un filtro solo para quedarnos con los valores que coincidan con el id del negocio

#### Limpieza y Normalizacion

In [42]:
# Convertir la columna 'date' a tipo datetime
dfReviewYelp['date'] = pd.to_datetime(dfReviewYelp['date'])

# Crear nuevas columnas para año, mes y día
dfReviewYelp['year'] = dfReviewYelp['date'].dt.year
dfReviewYelp['month'] = dfReviewYelp['date'].dt.month
dfReviewYelp['day'] = dfReviewYelp['date'].dt.day

In [44]:
columnas_a_eliminar = ['review_id', 'useful', 'funny', 'cool','date']

# Eliminar las columnas especificadas
dfReviewYelp = dfReviewYelp.drop(columnas_a_eliminar, axis=1)

In [47]:
# Aplicar la limpieza de texto directamente sobre la columna 'text'
dfReviewYelp['text'] = dfReviewYelp['text'].str.lower().str.replace(r'[^a-z0-9\s]', '')

In [48]:
# Modificamos el tipo de dato a un valor flotante
dfReviewYelp['stars'] = dfReviewYelp['stars'].astype('float64')

In [49]:
# Se añade la columna 'source', con valor unico 'Y'
dfBusinessYelp['source']='Y'

Se añade la columna 'source' y se usa 'Y' para indicar que los datos provienen del conjunto de datos de Yelp.

## 4. Almacenamiento de Datos

In [89]:
dfBusinessYelp.to_csv('DataSets/Cleaned_Business_Y.csv', index=False)

In [52]:
dfReviewYelp.to_csv('DataSets/Cleaned_Review_Y.csv', index= False)

In [51]:
dfReviewYelp.head(3)

,user_id,business_id,stars,text,year,month,day
447,XF-Bs5-h0wHWVBfK7WDScA,wQnVwt98pXX0n0Szro_XGg,4.0,me and my mom usually go here a lot and are sa...,2019,9,22
2247,5_eZ1vGz04nRP42y9jqX9A,67QczBNcTAYq8alxqwiZ5g,2.0,so much for the sub of the day special. gone f...,2019,5,25
2429,pzgJ1swB1S1VLc6yjRu66Q,vLdELbSYtwKVpkl4ownLEA,1.0,i have been coming to this subway for a year. ...,2019,7,9
